In [1]:
# 1. MONTAR GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json
import os
from PIL import Image
from tqdm import tqdm

# Ruta al JSON de anotaciones
json_path = "/content/drive/MyDrive/Colab Notebooks/VPCIII/TFINAL/cocotext.v2.json"

# Carpeta de imágenes COCO
img_dir = "/content/drive/MyDrive/Colab Notebooks/VPCIII/TFINAL/train2014"

# Carpeta donde guardar recortes
out_dir = "/content/drive/MyDrive/Colab Notebooks/VPCIII/TFINAL/dataset_recortes"
os.makedirs(out_dir, exist_ok=True)

# Cargar anotaciones
with open(json_path, "r") as f:
    data = json.load(f)

annotations = data["anns"]
imgs_info = data["imgs"]

print(f"Total de anotaciones: {len(annotations)}")

n_guardadas = 0

# Recorrer anotaciones
for ann_id, ann in tqdm(annotations.items()):
    if ann["legibility"] != "legible":
        continue
    if ann["utf8_string"] is None or ann["utf8_string"].strip() == "":
        continue

    img_id = ann["image_id"]
    img_info = imgs_info[str(img_id)]
    if img_info["set"] != "train":
        continue  # podemos cambiar esto si queremos recortes de 'val'

    image_filename = img_info["file_name"]
    image_path = os.path.join(img_dir, image_filename)

    if not os.path.exists(image_path):
        continue

    # Abrir imagen y recortar
    try:
        with Image.open(image_path).convert("RGB") as img:
            x, y, w, h = ann["bbox"]
            cropped = img.crop((x, y, x + w, y + h))
            label = ann["utf8_string"].strip()

            # Guardar recorte y label
            base_name = f"{img_id}_{ann_id}"
            cropped.save(os.path.join(out_dir, base_name + ".jpg"))
            with open(os.path.join(out_dir, base_name + ".txt"), "w", encoding="utf-8") as f:
                f.write(label)
            n_guardadas += 1
    except Exception as e:
        print(f"Error con imagen {image_path}: {e}")

print(f"✅ Recortes guardados: {n_guardadas}")


Total de anotaciones: 201126


100%|██████████| 201126/201126 [40:26<00:00, 82.88it/s] 

✅ Recortes guardados: 40611


In [ ]:
# Carpeta de imágenes COCO
img_dir_val = "/content/drive/MyDrive/Colab Notebooks/VPCIII/TFINAL/val2014"

# Carpeta donde guardar recortes
out_dir_val = "/content/drive/MyDrive/Colab Notebooks/VPCIII/TFINAL/dataset_recortes_val"
os.makedirs(out_dir_val, exist_ok=True)

# Cargar anotaciones
with open(json_path, "r") as f:
    data = json.load(f)

annotations = data["anns"]
imgs_info = data["imgs"]

print(f"Total de anotaciones: {len(annotations)}")

n_guardadas = 0

# Recorrer anotaciones
for ann_id, ann in tqdm(annotations.items()):
    if ann["legibility"] != "legible":
        continue
    if ann["utf8_string"] is None or ann["utf8_string"].strip() == "":
        continue

    img_id = ann["image_id"]
    img_info = imgs_info[str(img_id)]
    if img_info["set"] != "train":
        continue  # podemos cambiar esto si queremos recortes de 'val'

    image_filename = img_info["file_name"]
    image_path = os.path.join(img_dir_val, image_filename)

    if not os.path.exists(image_path):
        continue

    # Abrir imagen y recortar
    try:
        with Image.open(image_path).convert("RGB") as img:
            x, y, w, h = ann["bbox"]
            cropped = img.crop((x, y, x + w, y + h))
            label = ann["utf8_string"].strip()

            # Guardar recorte y label
            base_name = f"{img_id}_{ann_id}"
            cropped.save(os.path.join(out_dir, base_name + ".jpg"))
            with open(os.path.join(out_dir, base_name + ".txt"), "w", encoding="utf-8") as f:
                f.write(label)
            n_guardadas += 1
    except Exception as e:
        print(f"Error con imagen {image_path}: {e}")

print(f"✅ Recortes guardados: {n_guardadas}")

Total de anotaciones: 201126


100%|██████████| 201126/201126 [23:43<00:00, 141.32it/s]

✅ Recortes guardados: 19767


In [4]:
import os
import random
import shutil

# Rutas
base = "/content/drive/MyDrive/Colab Notebooks/VPCIII/TFINAL"
src_dir = os.path.join(base, "dataset_recortes")
train_out = os.path.join(base, "train2014_recortes")
val_out = os.path.join(base, "val2014_recortes")

# Crear carpetas destino
os.makedirs(train_out, exist_ok=True)
os.makedirs(val_out, exist_ok=True)

# Obtener todos los pares válidos
all_samples = []
for file in os.listdir(src_dir):
    if file.endswith(".jpg"):
        base_name = file[:-4]
        txt_path = os.path.join(src_dir, base_name + ".txt")
        if os.path.exists(txt_path):
            all_samples.append(base_name)

# Mezclar y dividir
random.shuffle(all_samples)
split_idx = int(len(all_samples) * 0.9)
train_samples = all_samples[:split_idx]
val_samples = all_samples[split_idx:]

print(f"📦 Ejemplos en total: {len(all_samples)}")
print(f"📦 Indice de corte split_idx: {split_idx}")
print(f"📦 Ejemplos de entrenamiento: {len(train_samples)}")
print(f"📦 Ejemplos de validación: {len(val_samples)}")

📦 Ejemplos en total: 60378
📦 Indice de corte split_idx: 54340
📦 Ejemplos de entrenamiento: 54340
📦 Ejemplos de validación: 6038


In [5]:
# Función para copiar pares
def copiar(samples, destino):
    for name in samples:
        shutil.copy(os.path.join(src_dir, name + ".jpg"), os.path.join(destino, name + ".jpg"))
        shutil.copy(os.path.join(src_dir, name + ".txt"), os.path.join(destino, name + ".txt"))

# Copiar
#copiar(train_samples, train_out)
copiar(val_samples, val_out)

#print(f"✅ Ejemplos entrenamiento: {len(train_samples)}")
print(f"✅ Ejemplos validación: {len(val_samples)}")
#print(f"Archivos copiados a:\n📁 {train_out}\n📁 {val_out}")
print(f"Archivos copiados a:\n📁 {val_out}")


KeyboardInterrupt: 

In [6]:
# Verificar si hay archivos en val2014_recortes
val_imgs = [f for f in os.listdir(val_out) if f.endswith(".jpg")]
val_txts = [f for f in os.listdir(val_out) if f.endswith(".txt")]

print(f"📦 Archivos .jpg en val2014_recortes: {len(val_imgs)}")
print(f"📄 Archivos .txt en val2014_recortes: {len(val_txts)}")

📦 Archivos .jpg en val2014_recortes: 3471
📄 Archivos .txt en val2014_recortes: 3470


In [ ]:
copiar(val_samples, val_out)

In [ ]:
validacion_recortes = os.path.join(base, "val2014_recortes")
copiar(val_samples, validacion_recortes)

In [ ]:
import os
import shutil
from tqdm import tqdm

val_out = "/content/drive/MyDrive/Colab Notebooks/VPCIII/TFINAL/val2014_recortes"
src_dir = "/content/drive/MyDrive/Colab Notebooks/VPCIII/TFINAL/dataset_recortes"

# Verificar si hay archivos en val2014_recortes
val_imgs = [f for f in os.listdir(val_out) if f.endswith(".jpg")]
val_txts = [f for f in os.listdir(val_out) if f.endswith(".txt")]

print(f"📦 Archivos .jpg en val2014_recortes: {len(val_imgs)}")
print(f"📄 Archivos .txt en val2014_recortes: {len(val_txts)}")


📦 Archivos .jpg en val2014_recortes: 6038
📄 Archivos .txt en val2014_recortes: 6038


In [ ]:
# Si está vacío, forzar la copia nuevamente (solo .jpg + .txt de val_samples)
if len(val_imgs) == 0 or len(val_txts) == 0:
    print("🔁 Reintentando copia de validación...")

    # Este bloque requiere que aún tengas `val_samples` definido. Si no:
    # 1. Volvé a cargar la lista `all_samples`
    # 2. Recalculá `val_samples = all_samples[split_idx:]`
    val_samples = all_samples[split_idx:]
    print(f"📦 Ejemplos de validación: {len(val_samples)}")
else:
    print("✅ val2014_recortes contiene archivos.")

In [ ]:
# Aquí suponemos que val_samples todavía existe
    for name in tqdm(val_samples, desc="Copiando val"):
        try:
            shutil.copy(os.path.join(src_dir, name + ".jpg"), os.path.join(val_out, name + ".jpg"))
            shutil.copy(os.path.join(src_dir, name + ".txt"), os.path.join(val_out, name + ".txt"))
        except Exception as e:
            print(f"❌ Error con {name}: {e}")